In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd

from plotting.constants import PROJECT_NAME_BASE_BO
from plotting.utils import get_runs, get_metrics_and_colors

In [3]:
metric = "fn_max"
lengthscale = 3.0

In [4]:
runs = get_runs(PROJECT_NAME_BASE_BO + str(lengthscale), mode="bo")

# Split runs based on seeds
runs_split = {}
for run in runs:
    seed_key = f"Seed {run.run.config['seed']}"
    if seed_key not in runs_split:
        runs_split[seed_key] = []
    runs_split[seed_key].append(run)

In [ ]:
# Get percentage improvement over random search for each seed + optimizer combo
percent_improvement = {}
for seed_key, runs in runs_split.items():
    metrics_dict, _ = get_metrics_and_colors(runs, metric)

    random_search_baseline = metrics_dict["Random Search"][0].metric_data[-1]
    for key, data in metrics_dict.items():
        if len(data) != 1:
            raise ValueError(
                f"Expected only one run for {key}, but got {len(data)}"
            )
        
        if key == "Random Search":
            continue
        # Calculate percent improvement
        if key not in percent_improvement:
            percent_improvement[key] = []
        percent_improvement[key].append(
            (data[0].metric_data[-1] - random_search_baseline) /
            random_search_baseline * 100
        )

print(percent_improvement)

{'PCG': [np.float64(12.173811979903853), np.float64(0.0), np.float64(7.170821371286662), np.float64(8.34721341026919), np.float64(0.0)], '\\texttt{ADASAP}': [np.float64(12.384485677750662), np.float64(7.98960866170164), np.float64(21.54122963064356), np.float64(13.465696095365235), np.float64(13.926247773579927)], '\\texttt{ADASAP-I}': [np.float64(9.753290776955584), np.float64(4.98972561317256), np.float64(17.180101981218538), np.float64(13.399674830593678), np.float64(11.004055354980528)], 'SDD-1': [np.float64(10.107639502263398), np.float64(5.891219123477629), np.float64(16.723147081329223), np.float64(13.366917715594855), np.float64(9.755035091622846)], 'SDD-10': [np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0)]}


In [6]:
# Average percent improvement over all optimizers
avg_percent_improvement = {}
for key, values in percent_improvement.items():
    avg_percent_improvement[key] = np.mean(values)

In [7]:
print(avg_percent_improvement)

{'PCG': np.float64(5.538369352291942), '\\texttt{ADASAP}': np.float64(13.861453567808207), '\\texttt{ADASAP-I}': np.float64(11.265369711384178), 'SDD-1': np.float64(11.16879170285759), 'SDD-10': np.float64(0.0)}
